<p style="font-size:30px; text-align:center; line-height:120%">
<br>
COMS W 4111-02, H02, V02<br>
Homework 4
</p>

# Setup

- Setting Up the Environment
    - Environment:
        - You will need to use Neo4j and Redis to complete this homework assignment.
        - Neo4j
            - Please follow the instructions in the previous [recorded makeup lecture](https://cvn.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=169786d3-c838-4f2c-ada6-ab1800d0e654) for setting up a cloud version of Neo4j.
            - You must have installed the sample movie database. The recorded lecture has instructions.
            - You must install the [Neo4j REST Client](https://docs.graphenedb.com/docs/python#section-neo4j-rest-client). The easiest way to do this is:
                - Go to the ```File``` menu option in the Notebook and select ```Open.``` This will open a new file explorer.
                - On the upper right hand corner, click on ```New``` and choose terminal. This will open a terminal window.
                - In the terminal window, enter ```pip install neo4jrestclient```
        - Redis:
            - Please follow the instructions in the [recorded makeup lecture](https://cvn.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=169786d3-c838-4f2c-ada6-ab1800d0e654) for setting up a cloud version of Redis.
            - You do not need to install or configure any data.
            - The Anaconda environment comes with a Redis client.
    - Section 2 tests your setup.

# Environment Test


In [194]:
import json
import pandas

## Neo4j

In [195]:
import neo4jrestclient
from neo4jrestclient.client import GraphDatabase

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

neo4jrestclient.options.VERIFY_SSL = False 

neo4j_host = "https://hobby-goabopedajfpgbkekbmjledl.dbs.graphenedb.com:24780/db/data/"
neo4j_user = "dff"
neo4j_password = "b.gbtWqr0pUhGH.pXp6S2jMfYf252w1"

gdb = GraphDatabase(neo4j_host, username = neo4j_user, password = neo4j_password)

In [196]:
def get_stuff(d):
    result = {}
    result['id'] = d['metadata']['id']
    result['labels'] = d['metadata'].get('labels', None)
    result['properties'] = d['data']
    return result


__Testing Getting Tom Hanks__

In [197]:
result = gdb.query("MATCH (n:Person) where n.name='Tom Hanks' return n")
for record in result:
    n = get_stuff(record[0])
    print("N = ", json.dumps(n, indent=2))

N =  {
  "id": 71,
  "labels": [
    "Person"
  ],
  "properties": {
    "born": 1956,
    "name": "Tom Hanks"
  }
}


__Testing a More Complex Thing__

In [12]:
result = gdb.query("MATCH (n:Person)-[r:ACTED_IN]->(m:Movie) where n.name='Tom Hanks' return n,r,m")
for record in result:
    actor = get_stuff(record[0])
    rel = get_stuff(record[1])
    movie = get_stuff(record[2])
    print("\tActor = ", json.dumps(actor))
    print("\tRelationship = ", json.dumps(rel))
    print("\tMovie = ", json.dumps(movie))
    print("")
    

	Actor =  {"id": 71, "labels": ["Person"], "properties": {"born": 1956, "name": "Tom Hanks"}}
	Relationship =  {"id": 202, "labels": null, "properties": {"roles": ["Jim Lovell"]}}
	Movie =  {"id": 144, "labels": ["Movie"], "properties": {"tagline": "Houston, we have a problem.", "title": "Apollo 13", "released": 1995}}

	Actor =  {"id": 71, "labels": ["Person"], "properties": {"born": 1956, "name": "Tom Hanks"}}
	Relationship =  {"id": 84, "labels": null, "properties": {"roles": ["Joe Fox"]}}
	Movie =  {"id": 67, "labels": ["Movie"], "properties": {"tagline": "At odds in life... in love on-line.", "title": "You've Got Mail", "released": 1998}}

	Actor =  {"id": 71, "labels": ["Person"], "properties": {"born": 1956, "name": "Tom Hanks"}}
	Relationship =  {"id": 234, "labels": null, "properties": {"roles": ["Jimmy Dugan"]}}
	Movie =  {"id": 162, "labels": ["Movie"], "properties": {"tagline": "Once in a lifetime you get a chance to do something different.", "title": "A League of Their Own

__Testing Some Cooler Stuff__

In [201]:
q =  """MATCH (n:Person)-[r:ACTED_IN]->(m:Movie) where n.name='Tom Hanks'
        return n.name,r.roles,m.title,m.released"""
result = gdb.query(q)
   
df = pandas.DataFrame(result, columns=["Name", "Roles", "Title", "Released"])    

df
    

,Name,Roles,Title,Released
0,Tom Hanks,[Jim Lovell],Apollo 13,1995
1,Tom Hanks,[Joe Fox],You've Got Mail,1998
2,Tom Hanks,[Jimmy Dugan],A League of Their Own,1992
3,Tom Hanks,[Joe Banks],Joe Versus the Volcano,1990
4,Tom Hanks,[Mr. White],That Thing You Do,1996
5,Tom Hanks,[Dr. Robert Langdon],The Da Vinci Code,2006
6,Tom Hanks,"[Zachry, Dr. Henry Goose, Isaac Sachs, Dermot ...",Cloud Atlas,2012
7,Tom Hanks,[Chuck Noland],Cast Away,2000
8,Tom Hanks,[Paul Edgecomb],The Green Mile,1999
9,Tom Hanks,[Sam Baldwin],Sleepless in Seattle,1993


## Redis

In [203]:
import redis

rd = redis.Redis(host="SG-W4111-28452.servers.mongodirector.com", password="FLGnHvHVnaG9Erq6tWOH1R92ip94608F")


data = json.dumps(list(result))
key = "Tom Hanks:ACTED_IN"
res = rd.set(key, data)

In [205]:
result = rd.keys("*")
print("Result = ", result)

Result =  [b'"Movie:{\\"released\\": 2008}"', b'Tom Hanks:ACTED_IN', b'"Movie:{\\"released\\": 2008, \\"title\\": \\"Frost/Nixon\\"}"', b'somestring', b'somejson']


In [206]:
roles = rd.get(key)
print("Raw answer = ", roles)
roles = json.loads(roles)
roles = pandas.DataFrame(roles, columns=["Name", "Roles", "Title", "Released"])

Raw answer =  b'[["Tom Hanks", ["Jim Lovell"], "Apollo 13", 1995], ["Tom Hanks", ["Joe Fox"], "You\'ve Got Mail", 1998], ["Tom Hanks", ["Jimmy Dugan"], "A League of Their Own", 1992], ["Tom Hanks", ["Joe Banks"], "Joe Versus the Volcano", 1990], ["Tom Hanks", ["Mr. White"], "That Thing You Do", 1996], ["Tom Hanks", ["Dr. Robert Langdon"], "The Da Vinci Code", 2006], ["Tom Hanks", ["Zachry", "Dr. Henry Goose", "Isaac Sachs", "Dermot Hoggins"], "Cloud Atlas", 2012], ["Tom Hanks", ["Chuck Noland"], "Cast Away", 2000], ["Tom Hanks", ["Paul Edgecomb"], "The Green Mile", 1999], ["Tom Hanks", ["Sam Baldwin"], "Sleepless in Seattle", 1993], ["Tom Hanks", ["Hero Boy", "Father", "Conductor", "Hobo", "Scrooge", "Santa Claus"], "The Polar Express", 2004], ["Tom Hanks", ["Rep. Charlie Wilson"], "Charlie Wilson\'s War", 2007]]'


In [207]:
roles

,Name,Roles,Title,Released
0,Tom Hanks,[Jim Lovell],Apollo 13,1995
1,Tom Hanks,[Joe Fox],You've Got Mail,1998
2,Tom Hanks,[Jimmy Dugan],A League of Their Own,1992
3,Tom Hanks,[Joe Banks],Joe Versus the Volcano,1990
4,Tom Hanks,[Mr. White],That Thing You Do,1996
5,Tom Hanks,[Dr. Robert Langdon],The Da Vinci Code,2006
6,Tom Hanks,"[Zachry, Dr. Henry Goose, Isaac Sachs, Dermot ...",Cloud Atlas,2012
7,Tom Hanks,[Chuck Noland],Cast Away,2000
8,Tom Hanks,[Paul Edgecomb],The Green Mile,1999
9,Tom Hanks,[Sam Baldwin],Sleepless in Seattle,1993


# HW4 $-$ Part 1: Neo4j

## Creating Nodes

- The following people are both ```Person``` and ```Fan.``` Please create nodes for these people.
    - George Washington
    - Thomas Jefferson
    - James Madison
    - Abraham Lincoln
    - John F. Kennedy
    

In [47]:
name = [
    "George Washington",
    "John Adams",
    "Thomas Jefferson",
    "James Madison",
    "James Monroe",
    "Ted Williams"
]
    
# Your create code goes here.

- After creation, run the following code. You should get the sample answer.

In [48]:
fan_q = "match (f:Fan) return f"
fans = []
fan_res = gdb.query(fan_q)

for f in fan_res:
    t = get_stuff(f[0])
    t = [t['id'], t['labels'], t['properties']['name']]
    fans.append(t)
    
fan_df = pandas.DataFrame(fans, columns=['Id', 'Labels', 'Name'])

In [45]:
fan_df

,Id,Labels,Name
0,171,"[Person, Fan]",John Adams
1,230,"[Person, Fan]",James Monroe
2,252,"[Person, Fan]",George Washington
3,271,"[Person, Fan]",Thomas Jefferson
4,272,"[Person, Fan]",Ted Williams
5,318,"[Person, Fan]",James Madison


## Creating Relationships

- Create ```Follows``` relationships:
    - John Adams ```Follows``` George Washing
    - Thomas Jefferson ```Follows``` John Adams
    - James Madison ```Follows``` John Adams
    - James Monroe ```Follows``` James Madison
    - EVERYONE ```Follows``` Ted Williams

In [49]:
# Your create code goes here.

    

- After running your code, returning all of the ```Fans``` in the browser should look like:

<img src="./fans.png">

- You can also run the following test to check your answer:

In [57]:
q = "match (n:Fan)-[r:FOLLOWS]->(m) return n.name, m.name"
df = pandas.DataFrame(d, columns=['Follower', 'Followee'])
df

,Follower,Followee
0,Thomas Jefferson,Ted Williams
1,Thomas Jefferson,John Adams
2,John Adams,Ted Williams
3,John Adams,George Washington
4,James Monroe,Ted Williams
5,James Monroe,James Madison
6,George Washington,Ted Williams
7,James Madison,Ted Williams
8,James Madison,Thomas Jefferson


- Create some ```Likes``` relationships:
    - Ted Williams ```Likes``` the ```Person``` Tom Hanks
    - James Monroe ```Likes``` the ```Person``` Robert Longo

In [61]:
# Your create code goes here.

- After creating the relationships, examing ```Likes``` in the browser should return:

<img src="./likes.png">



- The following code will test your result:

In [65]:
q = "match (n)-[:LIKES]->(m) return n.name, m.name"
res = gdb.query(q)
df_likes = pandas.DataFrame(res, columns=['Liker', 'Likee'])
df_likes

,Liker,Likee
0,Ted Williams,Tom Hanks
1,James Monroe,Robert Longo


## Spooky Query

- Find a shortest non-directed path from ```Person``` Robert Longo to ```Person``` Kevin Bacon. Running the query in the Neo4j browser will return:


<hr style="height:2px">
Write your query in here.

<hr style="height:2px">

- Running your query in Neo4j Browser will produce:

<img src="./path.png">

- Set the variable q below to your query.

In [103]:
q = "Enter your query here."

- You can test your query with the following code:

In [141]:
for p in res:
    for x in p:
        if type(x) == client.Relationship:
            n = x.start
            m = x.end
            l1 = n.get('name', None)
            if l1 is None:
                l1 = n.get('title', None)
            l2 = m.get('name', None)
            if l2 is None:
                l2 = m.get('title', None)
            print(l1, x.type, l2)
            

James Monroe LIKES Robert Longo
James Monroe FOLLOWS Ted Williams
Ted Williams LIKES Tom Hanks
Tom Hanks ACTED_IN Apollo 13
Kevin Bacon ACTED_IN Apollo 13


# Redis

## Introduction

- One if, if not the most common, uses of Redis is for result caching.


- The following figure provides an overview of the concept.

| <img src="./redis_cache.png"> |
| :---: |
| [Redis Caching](https://www.sohamkamani.com/blog/2016/10/14/make-your-node-server-faster-with-redis-cache/) |


- You can find more details on the concept [here](https://www.sohamkamani.com/blog/2016/10/14/make-your-node-server-faster-with-redis-cache/)


- We have seen the recurring pattern for queries of the form:

```
{
    "table_name": "t_name",
    "template": {
        "p1": "v1",
        "p2": "v1",
        ... ...
    }
}
```

- The concept represents simple queries in SQL, REST, ...


- The code below shows that you can do something similar with Neo4j.

In [177]:
def query_by_template(db, t_name, template):
    
    q = "match (n:" + t_name + ")  "
    
    terms = []
    for k,v in template.items():
        if type(v) == str:
            v = "'" + v + "'"
        terms.append("n."+ k + "=" + str(v))
        
    if len(terms) > 0:
        q += "where " + ' and '.join(terms)
        
    q += " return n "
    
    res = db.query(q)
    
    result = []
    for r in res:
        result.append(get_stuff(r[0]))
    return result

In [178]:
t_name = "Movie"
template = {"released": 2008}
res = query_by_template(gdb, t_name, template)
#print(res)

print(res)

[{'id': 121, 'labels': ['Movie'], 'properties': {'tagline': 'Speed has no limits', 'title': 'Speed Racer', 'released': 2008}}, {'id': 137, 'labels': ['Movie'], 'properties': {'tagline': '400 million people were waiting for the truth.', 'title': 'Frost/Nixon', 'released': 2008}}]


## Assignment

- We have seen that Redis stores data in many forms, one of which is key and string value.


- The following is a template for two functions that extend the example above to support caching in Redis

In [184]:
def cache_it(template, value):
    pass

def check_cache(template):
    pass

- And this is a simple example of extending the query function above to exploit caching.


In [190]:
import copy

def query_by_template_cache(db, t_name, template):
    
    if template is not None:
        tt = t_name + ":" + json.dumps(template)
    else:
        tt = t_name
        
    r = check_cache(tt)
    
    if r is not None:
        print("Cache HIT!")
        return json.loads(r)
    else:
        print("Cache MISS!")
    
    q = "match (n:" + t_name + ")  "
    
    terms = []
    for k,v in template.items():
        if type(v) == str:
            v = "'" + v + "'"
        terms.append("n."+ k + "=" + str(v))
        
    if len(terms) > 0:
        q += "where " + ' and '.join(terms)
        
    q += " return n "
    
    res = db.query(q)
    
    result = []
    for r in res:
        result.append(get_stuff(r[0]))
        
    cache_it(tt, json.dumps(result))
    
    return result

- Implement the functions ```check_cache``` and ```cache_hit.```


- You can test your functions below.

In [192]:
t_name = "Movie"
template = {"released": 2008, "title": "Frost/Nixon"}
res = query_by_template_cache(gdb, t_name, template)
print(res)

Cache MISS!
[{'id': 137, 'labels': ['Movie'], 'properties': {'tagline': '400 million people were waiting for the truth.', 'title': 'Frost/Nixon', 'released': 2008}}]


In [193]:
t_name = "Movie"
template = {"released": 2008, "title": "Frost/Nixon"}
res = query_by_template_cache(gdb, t_name, template)
print(res)

Cache HIT!
[{"id": 137, "labels": ["Movie"], "properties": {"tagline": "400 million people were waiting for the truth.", "title": "Frost/Nixon", "released": 2008}}]
